## Building RNN model for classification

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


### Importing required libraries and functions

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 3.6 MB/s 


In [ ]:
from imutils import paths
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import pickle
import math
import time

import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

### Restoring extracted features

In [ ]:
!cp /content/gdrive/MyDrive/train_features.tar.gz /content
!cp /content/gdrive/MyDrive/test_features.tar.gz /content
!cp /content/gdrive/MyDrive/class_name_list.pickle /content

In [ ]:
!tar xf train_features.tar.gz
!tar xf test_features.tar.gz

In [ ]:
with open('train_data.pickle', 'rb') as f:
  train_data = pickle.load(f)

with open('train_labels.pickle', 'rb') as f:
  train_labels = pickle.load(f)

with open('test_data.pickle', 'rb') as f:
  test_data = pickle.load(f)

with open('test_labels.pickle', 'rb') as f:
  test_labels = pickle.load(f)

with open('class_name_list.pickle', 'rb') as f:
  class_name_list = pickle.load(f)

In [ ]:
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (9537, 20, 1280)
Frame masks in train set: (9537, 20)


In [ ]:
print(f"Frame features in test set: {test_data[0].shape}")

Frame features in test set: (3783, 20, 1280)


### Normalizing the features

In [ ]:
train_features = np.copy(train_data[0])
train_features_rs = train_features.reshape((train_features.shape[0]*train_features.shape[1]), train_features.shape[2])

test_features = np.copy(test_data[0])
test_features_rs = test_features.reshape((test_features.shape[0]*test_features.shape[1]), test_features.shape[2])

In [ ]:
# min-max normalization
min_max_scaler = preprocessing.MinMaxScaler()
train_features_rs = min_max_scaler.fit_transform(train_features_rs)
test_features_rs = min_max_scaler.transform(test_features_rs)

In [ ]:
train_features = train_features_rs.reshape(train_features.shape[0], train_features.shape[1], train_features.shape[2])
test_features = test_features_rs.reshape(test_features.shape[0], test_features.shape[1], test_features.shape[2])

In [ ]:
# check if normalization was successfull: False implies success
print(np.array_equal(train_data[0], train_features))
print(np.array_equal(test_data[0], test_features))

False
False


### Setting parameters

In [ ]:
IMG_SIZE = 224
MAX_FRAMES = 0    #0 implies get all frames
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1280

### Trying different model architectures

In [ ]:
# # Testing using TensorBoard

# # Defining our sequence model
# class_vocab = class_name_list

# # Choosing hyperparameters
# lstm_layers = [2]
# layer_sizes = [256]
# dropout_list = [0.0]
# dense_layers = [0]
# dense_sizes = [256]

# for lstm_layer in lstm_layers:
#   for layer_size in layer_sizes:
#       for dropout in dropout_list:
#         for dense_layer in dense_layers:
#           for dense_size in dense_sizes:
#             NAME = f"{lstm_layer}-lstm-{layer_size}-nodes-{dropout}-dropout-{dense_layer}-dense-{dense_size}-densesize-{int(time.time())}"

#             frame_features_input = Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#             mask_input = Input((MAX_SEQ_LENGTH,), dtype="bool")

#             x = LSTM(512, return_sequences=bool(lstm_layer-1))(
#                 frame_features_input, mask=mask_input
#             )
#             x = Dropout(dropout)(x)
#             x = BatchNormalization()(x)

#             ret_seq = True
#             for l in range(lstm_layer-1):   # For adding n lstm layers in nth iteration
#               if (l==(lstm_layer-2)):
#                 ret_seq = False
#               x = LSTM(layer_size, return_sequences=ret_seq)(x)
#               x = Dropout(dropout)(x)
#               x = BatchNormalization()(x)

#             for l in range(dense_layer):
#               x = Dense(dense_size, activation="relu")(x)
#               x = Dropout(dropout)(x)

#             output = Dense(len(class_vocab), activation="softmax")(x)

#             rnn_model = Model([frame_features_input, mask_input], output)

#             opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)

#             rnn_model.compile(loss="sparse_categorical_crossentropy",
#                               optimizer=opt, 
#                               metrics=["accuracy"])
            
#             tensorboard = TensorBoard(log_dir=f'/content/logs/{NAME}')

#             early_stopper = EarlyStopping(patience=10)
            
#             filepath = "RNN-best-{val_accuracy:.2f}.hdf5"
#             checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor="val_accuracy", mode='max', verbose=1)

#             rnn_model.fit(
#               [train_features, train_data[1]],
#               train_labels,
#               validation_split=0.2,
#               epochs=EPOCHS,
#               # batch_size=BATCH_SIZE,
#               callbacks=[tensorboard, checkpoint, early_stopper],
#               )
            

In [ ]:
# # Testing using hyperparameters
# # Defining our sequence model
class_vocab = class_name_list

# NAME = f"3-lstm-512-256-256-nodes-0-dropout-0-dense-{int(time.time())}"

frame_features_input = Input((MAX_SEQ_LENGTH, NUM_FEATURES))
mask_input = Input((MAX_SEQ_LENGTH,), dtype="bool")

x = LSTM(2560, return_sequences=False)(
    frame_features_input, mask=mask_input
)
# x = BatchNormalization()(x)

# x = LSTM(640, return_sequences=True)(x)
# x = BatchNormalization()(x)

# x = LSTM(320, return_sequences=False)(x)
# x = BatchNormalization()(x)

# # for l in range(dense_layer):
# x = Dense(128, activation="relu")(x)
# x = Dropout(0.2)(x)

output = Dense(len(class_vocab), activation="softmax")(x)

rnn_model = Model([frame_features_input, mask_input], output)

opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

rnn_model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=opt, 
                  metrics=["accuracy"])

# tensorboard = TensorBoard(log_dir=f'/content/logs/{NAME}')

stopper = EarlyStopping(monitor='val_loss', patience=10)

# filepath = "RNN-best-{val_accuracy:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor="val_accuracy", mode='max', verbose=1)

rnn_model.fit(
  [train_features, train_data[1]],
  train_labels,
  validation_split=0.2,
  epochs=7,
  # batch_size=BATCH_SIZE,
  callbacks=[stopper]
  )


Epoch 1/7
239/239 [==============================] - 22s 76ms/step - loss: 1.7739 - accuracy: 0.5284 - val_loss: 8.4732 - val_accuracy: 0.0016
Epoch 2/7
239/239 [==============================] - 17s 72ms/step - loss: 0.7406 - accuracy: 0.7745 - val_loss: 10.2587 - val_accuracy: 0.0026
Epoch 3/7
239/239 [==============================] - 17s 72ms/step - loss: 0.4199 - accuracy: 0.8706 - val_loss: 10.0742 - val_accuracy: 0.0042
Epoch 4/7
239/239 [==============================] - 17s 72ms/step - loss: 0.2569 - accuracy: 0.9234 - val_loss: 9.9738 - val_accuracy: 0.0047
Epoch 5/7
239/239 [==============================] - 17s 71ms/step - loss: 0.2011 - accuracy: 0.9394 - val_loss: 10.7608 - val_accuracy: 0.0042
Epoch 6/7
239/239 [==============================] - 17s 71ms/step - loss: 0.1725 - accuracy: 0.9522 - val_loss: 10.7319 - val_accuracy: 0.0042
Epoch 7/7
239/239 [==============================] - 17s 72ms/step - loss: 0.1224 - accuracy: 0.9650 - val_loss: 10.6489 - val_accuracy: 0

In [ ]:
# Open tensorboard
%load_ext tensorboard
%tensorboard --logdir='logs/'
# # %reload_ext tensorboard

In [ ]:
# # Testing using Keras Tuner
LOG_DIR = f"{int(time.time())}"

class_vocab = class_name_list

def build_model(hp):
  frame_features_input = Input((MAX_SEQ_LENGTH, NUM_FEATURES))
  mask_input = Input((MAX_SEQ_LENGTH,), dtype="bool")
   
  x = LSTM(hp.Int('lstm_0_units', min_value=320, max_value=640, step=32), return_sequences=True)(
      frame_features_input, mask=mask_input
  )
  # x = Dropout(dropout)(x)
  # x = BatchNormalization()(x)

  # for i in range(hp.Int('n_lstm_layers', 0, 2)):
  x = LSTM(hp.Int(f'lstm_1_units', min_value=320, max_value=640, step=32), return_sequences=True)(x)
    # x = Dropout(dropout)(x)
    # x = BatchNormalization()(x)

  x = LSTM(hp.Int('lstm_2_units', min_value=320, max_value=640, step=32), return_sequences=False)(x)
  # x = Dropout(dropout)(x)
  # x = BatchNormalization()(x)

  # for i in range(hp.Int('n_d_layers', 0, 2)):
    # x = Dense(hp.Int(f'dense_{i}_units', min_value=64, max_value=512, step=32), activation="relu")(x)
  # x = Dropout(dropout)(x)

  output = Dense(len(class_vocab), activation="softmax")(x)

  rnn_model = Model([frame_features_input, mask_input], output)

  rnn_model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=Adam(learning_rate=1e-4), 
                  metrics=["accuracy"])

  return rnn_model

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory='LOG_DIR',
                     project_name='RNN_trials')

tuner.search(x=[train_features, train_data[1]], 
             y=train_labels, 
             epochs=600, 
             validation_split=0.2, 
             callbacks=[stop_early],
             verbose=1)


Trial 90 Complete [00h 02m 32s]
val_accuracy: 0.0041928719729185104

Best val_accuracy So Far: 0.006289307959377766
Total elapsed time: 02h 13m 31s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in LOG_DIR/RNN_trials
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
lstm_0_units: 352
lstm_1_units: 384
lstm_2_units: 640
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 3
tuner/round: 2
tuner/trial_id: b74f7c130a7f476281f853f28cbc1aac
Score: 0.006289307959377766
Trial summary
Hyperparameters:
lstm_0_units: 384
lstm_1_units: 416
lstm_2_units: 384
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.006289307959377766
Trial summary
Hyperparameters:
lstm_0_units: 480
lstm_1_units: 544
lstm_2_units: 384
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: bbe8f9943ef4caf85c7c10ce825eba2c
Score: 0.006289307959377766
Trial summary
Hyperparameters:
lstm_0_units: 512
lstm_1_units: 416
lstm_2_units: 576
tuner/epochs: 30
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.006289307959377766
Trial summary
Hyperparameters:
lstm_0_units: 3

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=3)[2]
best_hps.values

{'lstm_0_units': 480,
 'lstm_1_units': 544,
 'lstm_2_units': 384,
 'tuner/bracket': 2,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/round': 1,
 'tuner/trial_id': 'bbe8f9943ef4caf85c7c10ce825eba2c'}

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit([train_features, train_data[1]], 
                    train_labels, 
                    epochs=15, 
                    validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/15
239/239 [==============================] - 34s 85ms/step - loss: 2.2634 - accuracy: 0.4964 - val_loss: 8.2452 - val_accuracy: 0.0010
Epoch 2/15
239/239 [==============================] - 14s 59ms/step - loss: 0.5970 - accuracy: 0.8632 - val_loss: 9.1118 - val_accuracy: 0.0073
Epoch 3/15
239/239 [==============================] - 14s 59ms/step - loss: 0.2615 - accuracy: 0.9495 - val_loss: 9.4315 - val_accuracy: 0.0063
Epoch 4/15
239/239 [==============================] - 14s 59ms/step - loss: 0.1165 - accuracy: 0.9847 - val_loss: 9.6848 - val_accuracy: 0.0047
Epoch 5/15
239/239 [==============================] - 14s 59ms/step - loss: 0.0518 - accuracy: 0.9958 - val_loss: 10.0662 - val_accuracy: 0.0058
Epoch 6/15
239/239 [==============================] - 14s 59ms/step - loss: 0.0234 - accuracy: 0.9995 - val_loss: 10.3067 - val_accuracy: 0.0058
Epoch 7/15
239/239 [==============================] - 14s 59ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 10.5054 - val_accura

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit([train_features, train_data[1]], 
                train_labels, 
                epochs=best_epoch, 
                validation_split=0.2)

Epoch 1/2
239/239 [==============================] - 34s 83ms/step - loss: 2.2315 - accuracy: 0.5125 - val_loss: 8.3527 - val_accuracy: 0.0042
Epoch 2/2
239/239 [==============================] - 14s 59ms/step - loss: 0.5886 - accuracy: 0.8660 - val_loss: 8.5820 - val_accuracy: 0.0031


In [ ]:
# Output accuracy on test set
_, accuracy = hypermodel.evaluate([test_features, test_data[1]], test_labels)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

119/119 [==============================] - 3s 21ms/step - loss: 2.7030 - accuracy: 0.5358
Test accuracy: 53.58%


In [ ]:
hypermodel.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 20, 1280)]   0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 lstm_15 (LSTM)                 (None, 20, 480)      3381120     ['input_13[0][0]',               
                                                                  'input_14[0][0]']               
                                                                                                  
 lstm_16 (LSTM)                 (None, 20, 544)      2230400     ['lstm_15[0][0]']          

In [ ]:
hypermodel.save("hypermodel_3.h5")

In [ ]:
!cp /content/hypermodel_3.h5 /content/gdrive/MyDrive/

In [ ]:
with open('tuner_2.pickle', 'wb') as f:
  pickle.dump(tuner, f)

!cp /content/tuner_2.pickle /content/gdrive/MyDrive/

In [ ]:
# # Tune more hyperparameters using Keras Tuner

LOG_DIR = f"{int(time.time())}"

class_vocab = class_name_list

def build_model(hp):
  batchnorm = hp.Boolean('batch_norm')
  frame_features_input = Input((MAX_SEQ_LENGTH, NUM_FEATURES))
  mask_input = Input((MAX_SEQ_LENGTH,), dtype="bool")
   
  x = LSTM(480, return_sequences=True)(
      frame_features_input, mask=mask_input
  )
  x =  Dropout(hp.Float('dropout_rate_0', min_value=0, max_value=0.4, step=0.2))(x)
  if (batchnorm):
    x = BatchNormalization()(x)

  x = LSTM(544, return_sequences=True)(x)
  x =  Dropout(hp.Float('dropout_rate_1', min_value=0, max_value=0.4, step=0.2))(x)
  if (batchnorm):
    x = BatchNormalization()(x)

  x = LSTM(384, return_sequences=False)(x)
  x =  Dropout(hp.Float('dropout_rate_2', min_value=0, max_value=0.4, step=0.2))(x)
  if (batchnorm):
    x = BatchNormalization()(x)

  output = Dense(len(class_vocab), activation="softmax")(x)

  rnn_model = Model([frame_features_input, mask_input], output)

  rnn_model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=Adam(learning_rate=1e-4), 
                  metrics=["accuracy"])

  return rnn_model

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='LOG_DIR',
                     project_name='RNN_trials_bhp')

tuner.search(x=[train_features, train_data[1]], 
             y=train_labels, 
             epochs=150, 
             validation_split=0.2, 
             callbacks=[stop_early],
             verbose=1)


Trial 30 Complete [00h 01m 55s]
val_accuracy: 0.0041928719729185104

Best val_accuracy So Far: 0.006289307959377766
Total elapsed time: 00h 54m 39s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in LOG_DIR/RNN_trials_bhp
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
batch_norm: False
dropout_rate_0: 0.4
dropout_rate_1: 0.4
dropout_rate_2: 0.2
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.006289307959377766
Trial summary
Hyperparameters:
batch_norm: True
dropout_rate_0: 0.4
dropout_rate_1: 0.2
dropout_rate_2: 0.2
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: a2fad4895f4446c07029ae0c500409cb
Score: 0.0057651991955935955
Trial summary
Hyperparameters:
batch_norm: False
dropout_rate_0: 0.2
dropout_rate_1: 0.0
dropout_rate_2: 0.0
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.0057651991955935955
Trial summary
Hyperparameters:
batch_norm: True
dropout_rate_0: 0.0
dropout_rate_1: 0.4
dropout_rate_2: 0.2
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.0057651991955935

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]
best_hps.values

{'batch_norm': False,
 'dropout_rate_0': 0.4,
 'dropout_rate_1': 0.4,
 'dropout_rate_2': 0.2,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0}

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit([train_features, train_data[1]], 
                    train_labels, 
                    epochs=15, 
                    validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/15
239/239 [==============================] - 37s 80ms/step - loss: 2.5797 - accuracy: 0.4268 - val_loss: 8.5007 - val_accuracy: 0.0042
Epoch 2/15
239/239 [==============================] - 14s 57ms/step - loss: 0.7937 - accuracy: 0.8085 - val_loss: 9.2797 - val_accuracy: 0.0042
Epoch 3/15
239/239 [==============================] - 14s 58ms/step - loss: 0.4094 - accuracy: 0.9085 - val_loss: 9.8600 - val_accuracy: 0.0047
Epoch 4/15
239/239 [==============================] - 14s 58ms/step - loss: 0.2266 - accuracy: 0.9533 - val_loss: 10.1821 - val_accuracy: 0.0042
Epoch 5/15
239/239 [==============================] - 14s 58ms/step - loss: 0.1321 - accuracy: 0.9768 - val_loss: 10.2705 - val_accuracy: 0.0052
Epoch 6/15
239/239 [==============================] - 14s 58ms/step - loss: 0.0805 - accuracy: 0.9886 - val_loss: 10.5610 - val_accuracy: 0.0047
Epoch 7/15
239/239 [==============================] - 14s 58ms/step - loss: 0.0486 - accuracy: 0.9949 - val_loss: 10.8735 - val_accur

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit([train_features, train_data[1]], 
                train_labels, 
                epochs=best_epoch, 
                validation_split=0.2)

Epoch 1/5
239/239 [==============================] - 34s 79ms/step - loss: 2.5802 - accuracy: 0.4183 - val_loss: 8.4154 - val_accuracy: 0.0016
Epoch 2/5
239/239 [==============================] - 14s 57ms/step - loss: 0.8459 - accuracy: 0.7971 - val_loss: 9.1525 - val_accuracy: 0.0016
Epoch 3/5
239/239 [==============================] - 14s 57ms/step - loss: 0.4313 - accuracy: 0.8947 - val_loss: 9.6854 - val_accuracy: 0.0047
Epoch 4/5
239/239 [==============================] - 14s 57ms/step - loss: 0.2399 - accuracy: 0.9522 - val_loss: 10.0624 - val_accuracy: 0.0031
Epoch 5/5
239/239 [==============================] - 14s 57ms/step - loss: 0.1450 - accuracy: 0.9752 - val_loss: 9.9975 - val_accuracy: 0.0042


In [ ]:
# Output accuracy on test set
_, accuracy = hypermodel.evaluate([test_features, test_data[1]], test_labels)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

119/119 [==============================] - 3s 24ms/step - loss: 2.9320 - accuracy: 0.5630
Test accuracy: 56.3%


In [ ]:
hypermodel.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 20, 1280)]   0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 lstm_20 (LSTM)                 (None, 20, 480)      3381120     ['input_15[0][0]',               
                                                                  'input_16[0][0]']               
                                                                                                  
 dropout_19 (Dropout)           (None, 20, 480)      0           ['lstm_20[0][0]']          

### Saving the best models

In [ ]:
hypermodel.save("hypermodel_4.h5")

In [ ]:
!cp /content/hypermodel_4.h5 /content/gdrive/MyDrive/

In [ ]:
with open('tuner_3.pickle', 'wb') as f:
  pickle.dump(tuner, f)

!cp /content/tuner_3.pickle /content/gdrive/MyDrive/